In [1]:
import data_fft_function
from data_fft_function import preprocess
from data_fft_function import perct_change
from data_fft_function import fftprocess
from data_fft_function import fourier
#from data_fft_function import save_multi_image
from numpy.fft import fft, ifft
import os
from matplotlib.backends.backend_pdf import PdfPages
import sys
import pathlib
import datetime
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
#import PyPDF2 as pyd
from PyPDF2 import PdfFileWriter
import canvas
import PyPDF2 as pyd
from PyPDF2 import PdfFileWriter, PdfFileReader
plt.rcParams.update({"text.usetex": True,"font.family": "sans-serif", "font.sans-serif": ["Computer Modern Sans Serif"]})



# Ensure path is referenced to this script's root
# thisdir = os.path.dirname(__file__)
thisdir = Path.cwd()
# os.chdir(thisdir)
os.chdir(sys.path[0])
print(sys.path[0])
print(os.getcwd())

figs_dir = os.path.join(thisdir, 'figs/')
print(figs_dir)
if not os.path.isdir(figs_dir):
    os.makedirs(figs_dir)


c:\Users\samue\Desktop\Machine Learning\Workspace\Linear  Regression\IA1\Vibration Analysis
c:\Users\samue\Desktop\Machine Learning\Workspace\Linear  Regression\IA1\Vibration Analysis
c:\Users\samue\Desktop\Machine Learning\Workspace\Linear  Regression\IA1\Vibration Analysis\figs/


In [2]:

# Define Sampling Rate
Fs = 1e3

# Raw Data
rawdata = 'csvs/Vibration Data.csv'
data_id, cols, inputdata = preprocess(rawdata)
#print(inputdata)

# # Raw Data
# rawdata = 'csvs/M0981310.CSV'
# data_id, cols, inputdata = preprocess(rawdata)
# print(inputdata)



# creating new features
Current_value = inputdata['Current']
Previous_value = inputdata['Previous']

inputdata.loc[inputdata['Previous'] == ' ', 'Previous'] = 0 # Fill the missing values empty old values with 0
inputdata.loc[inputdata['Previous'] == 'NaN', 'Previous'] = 0
New_Prev = Previous_value
New_Prev = pd.to_numeric(New_Prev) # Convert to float type
Previous_value = New_Prev

# Percent Change Calculation
Pctchange = perct_change(Current_value, Previous_value) 
#print(inputdata)
# add the percentage change Column
inputdata["% Change"] = Pctchange
Post_data = inputdata.to_csv('csvs\Post_data.csv', index=False)
Post_data = pd.DataFrame(inputdata)
print(Post_data)

['Equipment', 'Equipment ID', 'Current', 'Previous', 'Collection Date']
data size (rows,columns) (920, 5)
           Equipment Equipment ID  Current Previous  Collection Date  % Change
0    Air Compressors        30 KW   1.1200        0  3/31/2022 14:05       inf
1    Air Compressors        30 KW   0.0288        0  3/31/2022 14:05       inf
2    Air Compressors        30 KW   0.5700        0  3/31/2022 14:05       inf
3    Air Compressors        30 KW   0.4880        0  3/31/2022 14:05       inf
4    Air Compressors        30 KW   2.9800        0  3/31/2022 14:01       inf
..               ...          ...      ...      ...              ...       ...
915      Smokehouses      Wins 14   0.0427        0  3/31/2022 14:14       inf
916      Smokehouses      Wins 14   0.4030        0  3/31/2022 14:13       inf
917      Smokehouses      Wins 14   0.0187        0  3/31/2022 14:13       inf
918      Smokehouses      Wins 14   0.3970        0  3/31/2022 14:13       inf
919      Smokehouses     

In [3]:
# FFT, IFFT 
# Extracting few important columns
Current_Amps = Post_data['Current']
Previous_Amps = Post_data['Previous']
#t_stamp = Post_data['Collection Date'] = pd.to_datetime(Post_data['Collection Date'])
#print(t_stamp.dtype)
Equipment = Post_data['Equipment']
EquipmentID = Post_data['Equipment ID']
#PercentChange = Post_data['% Change']

# Obtain a list of unique values in a pandas DataFrame column
Equipments = Equipment.unique()
EquipmentIDs = EquipmentID.unique()
myFmt = mdates.DateFormatter('%Y-%m-%d %H:%M')
gkk = Post_data.sort_values(by='Collection Date', ascending = False).groupby(['Equipment ID', 'Equipment', 'Current'])
g = gkk.first()
#print(g)



In [4]:
# Fast Fourier Transform
# FFT or DFT --> Converts from time domain to frequency domain
# ifft --> The inverse Fourier transform converts a signal in the frequency domain ω to  time domain 
# FFT and ifft, Need Data
# Raw Data
rawdata = 'csvs/Vibration Data.csv'
data_id, cols, inputdata = fftprocess(rawdata)
indata = pd.DataFrame(inputdata)
curr = indata['Current'].to_numpy()
prev = indata['Previous']
ID = indata['Equipment ID']
equip = indata['Equipment'] 


#fft_dict = fourier(curr, Fs)
[X,Y,freq,N] = fourier(curr,Fs)
fft_dict = {'X': X, 'Y': Y, 'freq':freq, 'N':N}
#splitting dictionary into data columns
X = fft_dict['X']
Y = fft_dict['Y']
freq = fft_dict['freq']
N = fft_dict['N']
#adding the column to the initial input data
indata['fft'] = X
indata['ifft'] = Y
indata['freq'] = freq
f = indata
# Data transformation for the purpose of plotting 
fkk = f.groupby(['Equipment ID', 'Equipment', 'freq', 'Current'])
f_new = fkk.first()
print(f_new)
# print(freq)

# # Plot fft figures
# fig, axs = plt.subplots(2, figsize=(15, 8))
# fig.suptitle('Frequency Domain')

# axs[0].plot(freq, X)
# axs[1].plot(freq, Y)

# # set labels
# plt.setp(axs, xlabel='Frequency')
# plt.setp(axs, ylabel='Amplitude')



['Equipment', 'Equipment ID', 'Current', 'Previous']
data size (rows,columns) (920, 4)
                                               Previous
Equipment ID Equipment       freq      Current         
30 KW        Air Compressors  0.000000 1.1200          
                              0.001087 0.0288          
                              0.002174 0.5700          
                              0.003261 0.4880          
                              0.004348 2.9800          
...                                                 ...
Wins 14      Smokehouses     -0.005435 0.0427          
                             -0.004348 0.4030          
                             -0.003261 0.0187          
                             -0.002174 0.3970          
                             -0.001087 0.4100          

[920 rows x 1 columns]


C:\Users\samue\AppData\Local\Temp\ipykernel_1368\143153114.py:30: FutureWarning: Dropping invalid columns in DataFrameGroupBy.first is deprecated. In a future version, a TypeError will be raised. Before calling .first, select only columns which should be valid for the function.
  f_new = fkk.first()


In [5]:


# Plots
#fig,ax = plt.subplots(figsize=(8,6))
#with PdfPages('foo.pdf') as pdf:

# create  a pdf file object

with PdfPages(figs_dir + f"Vibration Analysis.pdf", metadata={'Title': 'Vibration Analysis'}) as pdf:
      for page in range(pdf.get_pagecount):
        current_page = pdf.getPage(page)
     
      for i, g in Post_data.groupby(['Equipment ID']):
            plt.figure(figsize=(15,8))
            fig = g.plot(x = 'Collection Date', y = 'Current' , label=str(i))
            plt.ylabel('Amps')
            plt.xlabel('TimeStamp')
            #plt.ylim(ymin = 0)
            plt.xticks(rotation=45, fontsize=6)
            plt.gca().xaxis.set_major_formatter(myFmt)
            plt.ion
            pdf.savefig()
            plt.close()
            plt.show(block=False)
          
      for i, f_new in indata.groupby(['Equipment ID']):
            fig, ax3 = plt.subplots()
            ax3.plot(f_new['freq'], f_new['fft'], color='b', marker='o', markerfacecolor='m', zorder=2.5, alpha=0.5, label='Discrete FT')
            #f_new.plot(freq, fft, color='r', marker='o', markerfacecolor='m', zorder=2.5, alpha=0.5)
            ax3.set_ylabel(f'Amplitude', color='r')
            ax3.legend(loc='upper left')
            ax4 = ax3.twinx()
            
#             #fig = f_new.plot(x = 'freq', y = 'ifft' , label=str(i))
            ax4.plot(f_new['freq'], f_new['ifft'], color='r', marker='o', markerfacecolor='m', zorder=2.5, alpha=0.5, label = 'Inverse DFT')
            #f_new.plot(freq, ifft, color='r', marker='o', markerfacecolor='m', zorder=2.5, alpha=0.5)
            ax4.set_ylabel(f'Amplitude', color='b')
            ax4.legend(loc='upper right')
            ax3.set_xlabel(f'Frequency')
            ax3.set_title(f"FFT and IFFT",color='k', weight='bold', size=10)
            plt.ion
            pdf.savefig()
            plt.close()
            plt.show(block=False)
  
            
            
      # for i, f_new in indata.groupby(['Equipment ID']):
      #           #pdf = PdfPages('fig.pdf')
      #       plt.figure(figsize=(15,8))
      #       fig = f_new.plot(x = 'freq', y = 'ifft' , label=str(i))
      #       plt.ylabel('Amplitude')
      #       plt.xlabel('Frequency')
      #       plt.ylim(ymin = 0)
      #       plt.xticks(rotation=0, fontsize=6)
      #       plt.ion
      #       pdf.savefig()
      #       # page = pdf.get_pagecount()
      #       # d = pdf.infodict()
      #       # print(d)
      #       plt.close()
      #       plt.show(block=False)        

# We can also set the file's metadata via the PdfPages object:
            d = pdf.infodict()
            #print(d)
            d['Title'] = 'Vibration Analysis'
            # d['Author'] = 'Smithfield Foods Sioux  City'
            # d['Subject'] = 'How to create a multipage pdf file and set its metadata'
            # d['Keywords'] = 'PdfPages multipage keywords author title subject'
            # d['CreationDate'] = datetime.datetime.today()
            #d['ModDate'] = datetime.datetime.today()


TypeError: 'method' object cannot be interpreted as an integer